In [1]:
import tkinter as tk
from tkinter import ttk
import socket
import threading
from Techniques.des_encry_decry import bin_to_hexa,encrypt,pad,decrypt
from Techniques.rc4 import rc4_encrypt,rc4_decrypt

In [2]:
class CryptoChatServer:
    def __init__(self, root):
        self.root = root
        self.root.title("Server")

        # Create a StringVar to store the selected crypto technique
        self.crypto_technique_var = tk.StringVar()

        # Create a dropdown menu for crypto techniques
        crypto_label = tk.Label(root, text="Crypto Technique:")
        crypto_label.grid(row=0, column=0, padx=10, pady=10, sticky="e")

        crypto_options = ["AES", "RSA", "DES", "RC-4", "ELGAMMAL"]  # Add more as needed
        crypto_dropdown = ttk.Combobox(root, textvariable=self.crypto_technique_var, values=crypto_options)
        crypto_dropdown.grid(row=0, column=1, padx=10, pady=10, sticky="w")
        crypto_dropdown.set(crypto_options[0])  # Set default value

        # Create a text box for messages
        message_label = tk.Label(root, text="Message:")
        message_label.grid(row=1, column=0, padx=10, pady=10, sticky="e")

        self.message_entry = tk.Entry(root, width=40)
        self.message_entry.grid(row=1, column=1, padx=10, pady=10, sticky="w")

        # Create a chat window
        self.chat_window = tk.Text(root, height=15, width=50, state=tk.DISABLED)
        self.chat_window.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

        # Create a send button
        send_button = tk.Button(root, text="Send", command=self.send_message)
        send_button.grid(row=3, column=0, columnspan=2, pady=10)

        # Create a socket
        self.my_socket = socket.socket()
        self.my_socket.bind(("localhost", 4345))
        self.conn = None

        # Start a thread to listen for incoming connections
        threading.Thread(target=self.connector_thread).start()

    def connector_thread(self):
        self.my_socket.listen(5)
        self.conn, addr = self.my_socket.accept()
        print("Device connected")

        threading.Thread(target=self.receive_messages).start()

    def receive_messages(self):
        while True:
            try:
                data = self.conn.recv(1024).decode('utf-8')
                print(data)
                if not data:
                    break

                crypto_technique, message = data.split(":", 1)
                # Perform crypto operations on the received message using the selected technique
                if crypto_technique == "DES":
                    # Add decryption logic here
                    decrypted_message = bin_to_hexa(decrypt(message,key= "0E329232EA6D0D73"))
                    self.display_message(f"{crypto_technique}: {decrypted_message}")
                if crypto_technique == "RC-4":
                    key = "SecretKey"
                    decrypted_message =rc4_decrypt(key, message)
                    self.display_message(f"{crypto_technique}: {decrypted_message}")

            except ConnectionResetError:
                print("Connection closed.")
                break

    def send_message(self):
        if self.conn:
            message = self.message_entry.get()
            crypto_technique = self.crypto_technique_var.get()

            # Perform crypto operations on the message using the selected technique
            if crypto_technique == "DES":
                plain_text=pad(message)
                key= "0F1571C947D9E859"
                key=pad(key)
                cipher_text=bin_to_hexa(encrypt(plain_text,key))
                self.display_message(f"{crypto_technique}: {cipher_text}")
                

            if crypto_technique == "RC-4":
                key = "SecretKey"
                cipher_text = rc4_encrypt(key, message)
                self.display_message(f"{crypto_technique}: {cipher_text}")

            
            

            # Send the message to the client
            data = f"{crypto_technique}:{cipher_text}"
            msg = bytes(data, 'utf-8')
            self.conn.send(msg)

    def display_message(self, message):
        self.chat_window.config(state=tk.NORMAL)
        self.chat_window.insert(tk.END, message + "\n")
        self.chat_window.config(state=tk.DISABLED)


if __name__ == "__main__":
    root = tk.Tk()
    app = CryptoChatServer(root)
    root.mainloop()

    

Device connected
No padding required
No padding required
Encryption
Message after initial permutation:  5A005A003CF03C0F
The converted 56bit key is:  01101000111111000100010010100001000100010011111010010110

Round: Left key part: Right key part: SubKey used:
01       3CF03C0F        BAD22845     7833C320DA70
02       BAD22845        99E9B723     2B1A74CA48D8
03       99E9B723        0BAE3B9E     8C78D881D31D
04       0BAE3B9E        42415649     1667789316A0
05       42415649        18B3FA41     CE5D01D80B25
06       18B3FA41        9616FE23     4BAB4D126A9C
07       9616FE23        67117CF2     09F48B713191
08       67117CF2        C11BFC09     710DEAA3202B
09       C11BFC09        887FBC6C     129AB83347C3
10       887FBC6C        600F7E8B     9C38661E8103
11       600F7E8B        F596506E     A26E4CC66544
12       F596506E        738538B8     48772468A3C8
13       738538B8        C6A62C4E     C09D79F0D40B
14       C6A62C4E        56B0BD75     C5E2634E162A
15       56B0BD75        75